In [1]:
# Importando bibliotecas

import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Definindo as configurações de conexão

db_name = "db_hotel"
db_user = "hotel"
db_password = "hotel"
db_host = "localhost"
db_port = "5439"
    
database_url = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Cria o engine do SQLAlchemy
engine = create_engine(database_url)


In [3]:
# Definindo tabelas envolvidas e convertendo para DataFrame

reservas = pd.read_sql_query("SELECT * FROM hotel.reservas", engine)
rede_hoteleira = pd.read_sql_query("SELECT * FROM hotel.rede_hoteleira", engine)
endereco = pd.read_sql_query('SELECT * FROM hotel.endereco;', engine)
quartos = pd.read_sql_query('SELECT* FROM hotel.quartos', engine)
clientes = pd.read_sql_query("SELECT * FROM hotel.clientes;", engine)
acompanhantes = pd.read_sql_query("SELECT * FROM hotel.acompanhantes;", engine)

In [4]:
# 1. Connsultando do faturamento previsto por diário para o Grand Plaza Hotel

try:       
    # Filtrando por rede_id
    rede_hoteleira_filtered_id3 = rede_hoteleira[rede_hoteleira['rede_id'] == 3]

    # Fazendo o join das tabelas
    merged_df = reservas.merge(rede_hoteleira_filtered_id3, on='rede_id')

    # Agrupando por 'nome' e calculando a soma dos preços
    result_df_faturamento_grand_plaza = merged_df.groupby('nome').agg(faturamento_previsto_por_diaria=('preco', 'sum')).reset_index()

    # Imprimindo o DataFrame resultante
    print(result_df_faturamento_grand_plaza)

except Exception as e:
    print(f"Erro ao executar a manipulação de dados: {e}")

                 nome  faturamento_previsto_por_diaria
0  Grand Plaza Hotels                            450.0


In [5]:
# 2. Consultando o preço médio da diária para todos os quartos individuais de todas as redes cadastradas

try:
    # Filtrando a tabela reservas para selecionar apenas os quartos do tipo suíte
    reservas_filtered = reservas[reservas['suite'] == True]
    
    # Agregando o dataframe para calcular o preço médio
    result_df_preco_medio = reservas_filtered['preco'].mean().round(2)
    
    # Imprimindo Dataframe resultante
    print(result_df_preco_medio)
    
except Exception as e:
    print(f"Erro ao executar a manipulção de dados: {e}")

 

163.33


In [6]:
#  3. Consultar rede hoteleira que contenha a palavra Resort no nome.

try:
    # Filtrando o Dataframe por nome da rede hoteleira
    rede_hoteleira_filt_nome_Resort = rede_hoteleira[rede_hoteleira['nome'].str.contains('Resort')]
    
    # Imprimindo resutlado
    print(rede_hoteleira_filt_nome_Resort)
    
except Exception as e:
    print(f"Erro ao executar a manipulação de dados: {e}")

   rede_id            nome            cnpj  classificacao
1        2  Sunset Resorts  98765432109876              4


In [18]:
# 4. Consulta de quartos no estado de São Paulo com quartos abaixo de R$100,00

try:
    # Fazendo joins entre os DataFrames
    merged_rede_endereco_quartos = rede_hoteleira.merge(endereco, on='rede_id').merge(quartos, on='rede_id')

    # Aplicando filtros
    precos_abaixo_100_SP = merged_rede_endereco_quartos[(merged_rede_endereco_quartos['preco'] < 100) & (merged_rede_endereco_quartos['estado'] == 'São Paulo')]

    # Selecionando colunas especificadas
    precos_abaixo_100_SP = precos_abaixo_100_SP[['nome', 'estado', 'preco']]
    
    # Imprimindo resultado
    print(precos_abaixo_100_SP)
    
except Exception as e:
    print(f"Erro ao executar consulta: {e}")

                 nome     estado  preco
0      Hotel Paradise  São Paulo   90.0
4  Grand Plaza Hotels  São Paulo   80.0
6   Mountain Retreats  São Paulo   80.0
7   Mountain Retreats  São Paulo   75.0


In [8]:
# 5. Consultar todos os clientes e mostrar os acompanhantes cadastrados de cada cliente

try:
    # Fazendo join entre os DataFrames
    merged_clientes_acompanhantes = clientes.merge(acompanhantes, on='cliente_id', how='left' ,suffixes=('_cliente', '_acompanhante'))
    
    # Preenchendo os valores nulos na coluna de acompanhantes com "Sem acompanhante cadastrado"
    merged_clientes_acompanhantes['nome_acompanhante'].fillna('Sem acompanhante cadastrado', inplace=True)
    
    # Filtrando colunas
    merged_clientes_acompanhantes_nomes = merged_clientes_acompanhantes[['nome_cliente', 'nome_acompanhante']]
    
    # Imprimindo
    print(merged_clientes_acompanhantes_nomes)
    
except Exception as e:
    print(f"Erro ao executar consulta: {e}")

      nome_cliente            nome_acompanhante
0    João da Silva                    Ana Clara
1   Maria Oliveira  Sem acompanhante cadastrado
2     Pedro Santos                     Anderson
3        Ana Silva  Sem acompanhante cadastrado
4   Carlos Pereira               Pedro Henrique
5    Juliana Souza  Sem acompanhante cadastrado
6  Fernanda Santos                   Inês Costa
7  Rafael Oliveira                Beatriz Silva


In [9]:
# 6. Consultar o hotel mais barato de São Paulo ou Rio de Janeiro com o menor valor de diária.

try:
    # Aplicando filtros    
    menor_preco_sp_ou_rj = merged_rede_endereco_quartos[(merged_rede_endereco_quartos['preco'] == merged_rede_endereco_quartos['preco'].min()) & ((merged_rede_endereco_quartos['estado'] == 'São Paulo') | (merged_rede_endereco_quartos['estado'] == 'Rio de Janeiro'))]
    
    # Selecionando colunas
    menor_preco_sp_ou_rj_nome_estado_preco = menor_preco_sp_ou_rj[['nome', 'estado', 'preco']]
    
    # Imprimindo Dataframe resultante
    print(menor_preco_sp_ou_rj_nome_estado_preco)
    
except Exception as e:
    print(f"Erro ao executar a consulta: {e}")

                nome     estado  preco
7  Mountain Retreats  São Paulo   75.0


In [10]:
# 7. Consultar as redes hoteleira por valor de diária em ordem decrescente

try:
    # Fazendo join entre os DataFrames
    merged_rede_quarto = rede_hoteleira.merge(quartos, on='rede_id')
    
    # Aplicante filtros
    rede_preco_desc = merged_rede_quarto.sort_values(by='preco', ascending=False)
    
    # Selecionando Colunas
    rede_preco_desc_filt_por_nome_e_preco = rede_preco_desc[['nome', 'preco']]
    
    # Imprimindo DataFrame resultante
    print(rede_preco_desc_filt_por_nome_e_preco)
    
except Exception as e:
    print(f"Erro ao executar a consulta: {e}")

                        nome  preco
5  Urban Heights Hotel Group  190.0
1             Sunset Resorts  120.0
0             Hotel Paradise   90.0
2         Grand Plaza Hotels   80.0
3          Mountain Retreats   80.0
4          Mountain Retreats   75.0


In [11]:
# 8.  Consultar todas as redes hoteleiras que possuem quartos com preço superior a média de preço.

try:
    # Calcular a média de preço por rede hoteleira
    media_preco_por_rede = quartos.groupby('rede_id')['preco'].mean().reset_index()
    media_preco_por_rede.rename(columns={'preco': 'media_preco'}, inplace=True)
    
    # Fazendo o merge para adicionar a média de preço ao DataFrame da rede_hoteleira
    rede_hoteleira_com_media = rede_hoteleira.merge(media_preco_por_rede, on='rede_id')
    
    # Filtrar os hotéis onde existe pelo menos um quarto com preço acima da média
    filt_hotel_preco_acima_media = rede_hoteleira_com_media[rede_hoteleira_com_media['rede_id'].isin(
        quartos[quartos['preco'] > quartos['rede_id'].map(media_preco_por_rede.set_index('rede_id')['media_preco'])]['rede_id']
    )]
    
    # Selecionar apenas o nome das redes hoteleiras
    rede_preco_acima_media = filt_hotel_preco_acima_media[['nome']]
    
    # Imprimir o resultado
    print(rede_preco_acima_media)
    
except Exception as e:
    print(f"Erro ao executar a query: {e}")

                nome
3  Mountain Retreats


In [12]:
# 9. Um ranking dos quartos tipo suíte, do mais caro para o mais barato, com base no preço médio.

try:
    # Filtrando as reservas por suite
    reservas_filtradas_suite = reservas[reservas['suite'] == True]
    
    # Calcular a média de preço por número de quarto
    media_preco_por_suite = reservas_filtradas_suite.groupby('numero_quarto')['preco'].mean().sort_values(ascending=False).reset_index()
    media_preco_por_suite.rename(columns={'preco': 'media_preco'}, inplace=True)
    
    # Calcular o ranking de preço por número de quarto
    media_preco_por_suite['ranking_preco'] = media_preco_por_suite['media_preco'].rank(method='dense', ascending=False)
    
    # Ajustando o tipo de dados da coluna ranking_preco para inteiro
    media_preco_por_suite['ranking_preco'] = media_preco_por_suite['ranking_preco'].astype(int)
    
    # Imprimir o resultado
    print(media_preco_por_suite)

except Exception as e:
    print(f"Erro ao executar a consulta: {e}")


  numero_quarto  media_preco  ranking_preco
0           115        190.0              1
1           105        150.0              2
